In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import scipy

In [ ]:
train_labels_2 = labelDf.loc[[int(star) for star in train_stars]]
m_star = train_labels_2.mean(axis=1)
s_star = train_labels_2.std(axis=1)
# normalize all stars such that they have a similar spectrum
for col in train_labels_2.columns:
    train_labels_2[col] = (train_labels_2[col] - m_star)/s_star

m_wl = train_labels_2.mean(axis=0)
s_wl = train_labels_2.std(axis=0)

#normalize every wavelengths such that it has values between -1,1
for col in train_labels_2.columns:
    train_labels_2[col] = (train_labels_2[col] - m_wl[col])/s_wl[col]

train_labels_2['s'] = abs(train_labels_2).sum(axis=1)
train_labels_2.s.std(), train_labels_2.s.mean(), train_labels_2.s.max(), train_labels_2

In [ ]:
if 0:
    labelDf = pd.read_csv("train_labels.csv")
    labelDf = labelDf.set_index('planet_id')
    meanLabels = np.mean(labelDf.mean())
    stdLabels = np.std(labelDf.std())
    maxLabels = np.max(labelDf.max())
    minLabels = np.min(labelDf.min())

    trainLabels = labelDf.loc[[int(star) for star in train_stars]].copy()
    meanTrainLabels = np.mean(trainLabels.mean())
    stdTrainLabels = np.std(trainLabels.std())
    maxTrainLabels = np.max(trainLabels.max())
    minTrainLabels = np.min(trainLabels.min())

    #scale up data
    for col in labelDf.columns:
        labelDf.loc[:,col] = (labelDf[col]) / (maxTrainLabels)

    m_star = labelDf.mean(axis=1)
    s_star = labelDf.std(axis=1)
    # normalize all stars such that they have a similar spectrum -> we need to predict those values as well
    for col in trainLabels.columns:
        #trainLabels[col] = (trainLabels[col] - m_star)/s_star
        labelDf[col] = (labelDf[col] - m_star)/s_star

    trainLabels = labelDf.loc[[int(star) for star in train_stars]]
    m_wl = trainLabels.mean(axis=0)
    s_wl = trainLabels.std(axis=0)

    #normalize every wavelengths such that it has values between -1,1 -> those are constants per wavelength
    for col in labelDf.columns:
        labelDf[col] = (labelDf[col] - m_wl[col])/s_wl[col]
    labelDf

In [ ]:
fig = go.Figure()
for i in [0,1,2,3]:
    fig.add_trace(go.Scatter(y=train_labels_2.iloc[i],mode='markers',marker=dict(size=3)))
fig.show()

In [463]:
train_stars_1, test_stars_1
import pickle
with open('train_stars_1.pkl', 'wb') as file:
    pickle.dump(train_stars_1, file)

with open('test_stars_1.pkl', 'wb') as file:
    pickle.dump(test_stars_1, file)

In [ ]:
len(train_stars), len(test_stars),list(set(test_stars) &set(train_stars_1)) # overlap of old training & new validation -> no old training in valiation

In [48]:
tf.random.set_seed(42)
files = glob.glob(os.path.join('train/', '*/*'))
stars = []
for file in files:
    file_name = file.split('\\')[1]
    stars.append(file_name)
stars = np.unique(stars)

import random
random.seed(42)

def split_star_list(file_list, test_ratio=0.1):
    random.shuffle(file_list)
    split_index = int(len(file_list) * (1 - test_ratio))
    train_files = file_list[:split_index]
    test_files = file_list[split_index:]
    return train_files, test_files

train_stars, test_stars = split_star_list(stars)

labelDf = pd.read_csv("train_labels.csv")
labelDf = labelDf.set_index('planet_id')
meanLabels = np.mean(labelDf.mean())
stdLabels = np.std(labelDf.std())
maxLabels = np.max(labelDf.max())
minLabels = np.min(labelDf.min())

trainLabels = labelDf.loc[[int(star) for star in train_stars]]
meanTrainLabels = np.mean(trainLabels.mean())
stdTrainLabels = np.std(trainLabels.std())
maxTrainLabels = np.max(trainLabels.max())
minTrainLabels = np.min(trainLabels.min())

for col in labelDf.columns:
    labelDf.loc[:,col] = (labelDf[col]) / (maxTrainLabels)

# normalize over time and all samples, so we have a mean and a std dev per wavelength for all samples
def calcMeanAndStdOfTrain(train_stars):
    i = 0
    for star in train_stars:
        file_path = 'train/'+str(star)+'/combined.npz'
        with np.load(file_path) as data:
            x = data['a'][0,:,0:283,:]
            if i ==0:
                mean = np.mean(x,axis=(0))
                sumS = np.sum(x**2,axis=0)
            else:
                mean = mean + np.mean(x, axis=(0))
                sumS += np.sum(x**2,axis=0)
            i=i+1
    meanTrain = mean / i
    stdTrain = np.sqrt(sumS / (i*x.shape[0]) - meanTrain**2)    
    return meanTrain, stdTrain
meanTrain, stdTrain = calcMeanAndStdOfTrain(train_stars)

def normalize_over_train(features, labels):
    #features = (features - meanTrain) / (stdTrain + 1e-6)
    features = (features - meanTrain) / (stdTrain + 1e-6)
    return features, labels

# normalize over time per samples, so we have a mean and a std dev per wavelength for all samples
def calcMeanAndStdOfTrainPerStar(x):
    mean = np.mean(x,axis=(0))
    sumS = np.sum(x**2,axis=0)
    stdTrain = np.sqrt(sumS / (x.shape[0]) - mean**2)    
    return mean, stdTrain
def normalize_per_sample(features, labels):
    m,s = calcMeanAndStdOfTrainPerStar(features)
    features = (features) / (s + 1e-6)
    return features, labels




def load_npz(star):
    integer_value = tf.strings.to_number(star, out_type=tf.int64)
    python_int = integer_value.numpy()

    file_path = 'train/'+str(python_int)+'/combined.npz'
    try:
        with np.load(file_path) as data:
            features = data['a'][0,:,0:283,:]
            labels = labelDf.loc[python_int].to_numpy()
            meanL = np.mean(labels)
            stdL = np.std(labels)
            labels = (labels-meanL)*100 / stdL
            features = np.reshape(features,(-1,25,283,4))
            features = np.mean(features,axis=1)
            #features, labels = normalize_per_sample(features,labels)
            features, labels = normalize_over_train(features,labels)
            return features, labels
    except Exception as e:
        print("Error loading file:", e, python_int)


def create_dataset(star_list, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices(star_list)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(star_list), reshuffle_each_iteration=True)
    def load_and_process(x):
        features, labels = tf.py_function(
            func=load_npz,
            inp=[x],
            Tout=[tf.float64, tf.float32]
        )
        return features, labels

    dataset = dataset.map(load_and_process, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x, y: (tf.ensure_shape(x,tf.TensorShape([225, 283, 4])), tf.ensure_shape(y, tf.TensorShape(283)))) #5625
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


In [49]:
tf.random.set_seed(42)
batch_size = 64

train_dataset = create_dataset(train_stars, batch_size, shuffle=True)
test_dataset = create_dataset(test_stars, batch_size, shuffle=False)

In [23]:
small_train = create_dataset(train_stars[0:batch_size], batch_size, shuffle=True)

In [ ]:
for x,y in train_dataset:
    print(y[0,0])
for x,y in test_dataset:
    print('test',y[0,0])

# model

In [ ]:
class Reshape1(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        x = tf.transpose(x, perm=[0,2,1,3])
        #x = tf.reshape(x, [-1, self.timepoints, tf.cast(self.wavelengths * self.representations, tf.int32)])
        return x
    
class Reshape11(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        x = tf.transpose(x, perm=[0,2,1])
        #x = tf.reshape(x, [-1, self.timepoints, tf.cast(self.wavelengths * self.representations, tf.int32)])
        return x

class Reshape2(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x_pred, x_confidence):
        x = tf.concat([x_pred, x_confidence], axis = -1)
        
        return x
    
class Reshape22(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x_pred, x_confidence):
        x_pred = tf.expand_dims(x_pred, axis=-1)
        x_confidence = tf.expand_dims(x_confidence, axis=-1)
        x = tf.concat([x_pred, x_confidence], axis = -1)
        return x
    
class Reshape3(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        x = tf.reshape(x, (None,-1,x.shape[2]))
        return x
    
class reduce(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        mean = tf.reduce_sum(x,axis=-1)
        mean = tf.expand_dims(mean, axis=-1)
        return mean
class reduce1(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        mean = tf.reduce_sum(x,axis=-1)
        return mean
    
class tile(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x,mean):
        x = tf.concat([x,mean],axis=-1)
        return x
    
class tile2(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x,mean):
        x = tf.concat([x,tf.expand_dims(mean,axis=-1)],axis=-2)
        return x
    
class meanOfWavelengths(tf.keras.layers.Layer):
    def __init__(self, concat=True,**kwargs):
        self.concat=concat
        super().__init__(**kwargs)
    def call(self, x):
        m = tf.expand_dims(tf.reduce_mean(x,axis=-1),axis=-1)
        x = tf.concat([x,m],axis=-1)
        return x if self.concat else m

# gated linear unit, splits input in 2 batches, second batch is activation
class GLU(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x, mask=None):
        x,gate = tf.split(x, 2, axis = -1)
        # swish = gate * sigmoid(gate) (sigmoid = between 0..1)
        x = x*tf.keras.activations.swish(gate) # use one input as a gate such that the network is able to focus on information
        return x

class GLUMlp(tf.keras.layers.Layer):
    def __init__(self, dim_expand, dim, **kwargs):
        super().__init__(**kwargs)
        self.dim_expand = dim_expand
        self.dim = dim
        # same operation as dense layer
        self.dense_1 = tf.keras.layers.EinsumDense("abc,cd->abd",output_shape=(None, self.dim_expand), activation = 'linear', bias_axes = 'd')
        self.glu_1 = GLU()
        self.dense_2 = tf.keras.layers.EinsumDense("abc,cd->abd",output_shape=(None, self.dim), activation = 'linear', bias_axes = 'd')
    def call(self, x, training = False):
        #print('glu_input',x.shape)
        x = self.dense_1(x)
        #print(x.shape)
        x = self.glu_1(x)
        #print(x.shape)
        x = self.dense_2(x)
        #print(x.shape)
        return x

class ScaleBias(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def build(self, input_shape):
        self.scale_bias = tf.keras.layers.EinsumDense("abc,c->abc",output_shape=(None, input_shape[-1]),activation = 'linear', bias_axes = 'c')
    def call(self, x, mask=None):
        return self.scale_bias(x)

#attention gets calculated along first dimension!
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim//num_heads)
        self.ffn = GLUMlp(feed_forward_dim, embed_dim)
        #self.ffn = tf.keras.layers.Dense(feed_forward_dim)
        self.layer_norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6) # normalization by a * (input-mean) /sqrt(var + eps) +b    where a and b are learned, eps is to avoid div/0
        self.layer_norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        #self.scale_bias_1 = ScaleBias()
        #self.scale_bias_2 = ScaleBias()
    def call(self, x, training = None):
        residual = x
        #print('before att')
        x = self.att(x, x)
        #x = self.scale_bias_1(x)
        x = self.layer_norm_1(x + residual)
        #x = x+residual
        residual = x
        #print('after att')
        x = self.ffn(x, training = training)
        #print('after glu')
        #x = self.scale_bias_2(x)
        x = self.layer_norm_2(x + residual)
        return x
    
# is effectively an attention mechanism to allow some columns to be used / turned off
# effective channel attention!
class ECA(tf.keras.layers.Layer):
    # TF implementation from https://www.kaggle.com/code/hoyso48/1st-place-solution-training
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False) # only one1D convolution with kernel size
    def call(self, inputs):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs) # works not on batch size, but on next dimension, e.g. batch_size, 60x9 -> works on 60, so output is batch_size x 9
        nn = tf.expand_dims(nn, -1) # a,c -> a,c,1
        nn = self.conv(nn) # a,c,1 -> a,c,1 (1, because conv is only having 1 filter)
        nn = tf.squeeze(nn, -1) # a,c,1 -> a,c
        nn = tf.nn.sigmoid(nn) # a,c -> a,c
        nn = nn[:,None,:] # a,1,c -> e.g. batch_size,1,9
        return inputs * nn # a,1,c * a,b,c applies broadcasting / elementwise multiplication -> turns input on or off column wise


class HeadDense(tf.keras.layers.Layer):
    def __init__(self, head_dim, **kwargs):
        super().__init__(**kwargs)
        self.head_dim = head_dim
    def build(self, input_shape):
        self.length = input_shape[1]
        self.dim = input_shape[2]
        self.dense = tf.keras.layers.EinsumDense("abc,cd->abd",output_shape=(self.length, self.head_dim), activation = 'swish', bias_axes = 'd') #siwsh is causing a self gating
    def call(self, x):
        x = self.dense(x)
        return x
    
class Conv1DBlockSqueezeformer(tf.keras.layers.Layer):
    def __init__(self, channel_size, kernel_size, dilation_rate=1,
                 expand_ratio=2, se_ratio=0.25, activation='swish', name=None, **kwargs):
        super().__init__()
        self.channel_size = channel_size
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate
        self.expand_ratio = expand_ratio
        self.se_ratio = se_ratio
        self.activation = activation
        self.scale_bias = ScaleBias()
        self.glu_layer = GLU()
        self.ffn = GLUMlp(channel_size*4, channel_size)
        self.layer_norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.scale_bias_1 = ScaleBias()
        self.scale_bias_2 = ScaleBias()
    def build(self, input_shape):
        self.length = input_shape[1]
        self.channels_in = input_shape[2]
        self.channels_expand = self.channels_in * self.expand_ratio
        self.dwconv = tf.keras.layers.DepthwiseConv1D(self.kernel_size,dilation_rate=self.dilation_rate,padding='same',use_bias=False)
        self.BatchNormalization_layer = tf.keras.layers.BatchNormalization(momentum=0.95)
        self.conv_activation = tf.keras.layers.Activation(self.activation)
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.ECA_layer = ECA() #convolutional attention
        self.expand = tf.keras.layers.EinsumDense("abc,cd->abd",output_shape=(self.length, self.channels_expand), activation = 'linear', bias_axes = 'd')
        self.project =tf.keras.layers.EinsumDense("abc,cd->abd",output_shape=(self.length, self.channel_size), activation = 'linear', bias_axes = 'd')
    def call(self, x, training = None):
        skip = x
        #print(x.shape)
        x = self.expand(x) #dense layer expands time dimension
        #print(x.shape)
        x = self.glu_layer(x) # gating of input through linear gating unit, 2 halfs, second half = activation of first(=input)
        #print('glu',x.shape)
        x = self.dwconv(x)
        #print('conv filter',x.shape)
        x = self.BatchNormalization_layer(x)
        #print('batchnorm',x.shape)
        x = self.conv_activation(x)
        #print('activation f',x.shape)
        x = self.ECA_layer(x) #conv attention
        #print('eca',x.shape)
        x = self.project(x)
        #print(x.shape)
        x = self.scale_bias_1(x)
        #print(x.shape)

        x = x+skip

        residual = x
        x = self.ffn(x) # ff + gate
        x = self.scale_bias_2(x)
        x = self.layer_norm_2(x + residual)
        return x

timepoints = 225
representations = 4
wavelengths = 283
targetWavelengths = 283

def fcn():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1000)(x)
    #x = tf.keras.layers.Dense(100)(x)
    
    x_pred = tf.keras.layers.Dense(283, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model

def cnn1():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    kernelS = 20
    for i in range(25):       
        x = tf.keras.layers.Conv1D(filters=283, kernel_size=(5), padding='valid')(x)
        #x = tf.keras.layers.AveragePooling1D(2)(x)
        x = Reshape11()(x) # wavelengths x timepoints
        x = tf.keras.layers.Dense(timeP)(x)  # dense on timepoints
        x = Reshape11()(x) # timepoints  *  wavelengths

    #x = tf.keras.layers.DepthwiseConv1D(kernel_size=5,strides=1,padding='same', depth_multiplier=1,activation='relu')(x)
    x = tf.keras.layers.Dense(283)(x)
    x = Reshape11()(x)
    #x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(100)(x)
    #x = tf.keras.layers.Dense(50)(x)
    
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model

def cnn2():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    kernelS = 20
    for i in range(5):
        # depthwise1d filter -> one filter per channel (=wavelength), depth_multiplier tells us how many filters per channel
        x = tf.keras.layers.DepthwiseConv1D(kernel_size=kernelS,strides=1,padding='valid', depth_multiplier=1,activation='relu')(x)
        timeP = timeP - kernelS
        
        #x = tf.keras.layers.Conv1D(filters=283, kernel_size=(20), padding='valid')(x)
        #x = tf.keras.layers.AveragePooling1D(2)(x)
        #x = Reshape11()(x) # wavelengths x timepoints
        #x = tf.keras.layers.Dense(timeP)(x)  # dense on timepoints
        #x = Reshape11()(x) # timepoints  *  wavelengths

    #x = tf.keras.layers.DepthwiseConv1D(kernel_size=5,strides=1,padding='same', depth_multiplier=1,activation='relu')(x)
    x = tf.keras.layers.Dense(283)(x)
    x = Reshape11()(x)
    #x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(100)(x)
    x = tf.keras.layers.Dense(50)(x)
    
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model

def cnn3():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    kernelS = 20
    n_conv=20
    depth_cnn = 7
    for j in range(depth_cnn):
        # depthwise1d filter -> one filter per channel (=wavelength), depth_multiplier tells us how many filters per channel
        
        out = []
        for i in range(5):
            x0 = tf.keras.layers.DepthwiseConv1D(kernel_size=kernelS,strides=1,padding='valid', depth_multiplier=1,activation='relu')(x)
            out.append(x0)
        x = tf.keras.layers.Concatenate(axis=-1)(out)
        x = tf.keras.layers.Dense(283)(x)
        print(x.shape)
        
        #x = tf.keras.layers.Conv1D(filters=283, kernel_size=(20), padding='valid')(x)
        #x = tf.keras.layers.AveragePooling1D(2)(x)
        #x = Reshape11()(x) # wavelengths x timepoints
        #x = tf.keras.layers.Dense(timeP)(x)  # dense on timepoints
        #x = Reshape11()(x) # timepoints  *  wavelengths

    #x = tf.keras.layers.DepthwiseConv1D(kernel_size=5,strides=1,padding='same', depth_multiplier=1,activation='relu')(x)
    print(x.shape)
    x = tf.keras.layers.Dense(283)(x)
    x = Reshape11()(x)
    #x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(100)(x)
    x = tf.keras.layers.Dense(50)(x)
    
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model

def cnn2D():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,0:1]
    for i in range(3):
        # depthwise1d filter -> one filter per channel (=wavelength), depth_multiplier tells us how many filters per channel
        #x = tf.keras.layers.DepthwiseConv1D(kernel_size=10,strides=1,padding='valid', depth_multiplier=1,activation='relu')(x)
        x = tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), padding='same')(x)
        x = tf.keras.layers.AveragePooling2D((2,1))(x)
        #x = tf.keras.layers.Dense(284)(x)

    #x = tf.keras.layers.DepthwiseConv1D(kernel_size=5,strides=1,padding='same', depth_multiplier=1,activation='relu')(x)
    #x = tf.keras.layers.Dense(283)(x)
    #x = Reshape3()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1000)(x)
    #x = tf.keras.layers.Dense(100)(x)
    
    x_pred = tf.keras.layers.Dense(283, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model

def squeezeformer():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    conv_filter = 15
    dim=timepoints
    x = TransformerEncoder(wavelengths, 4, wavelengths*4)(x)
    x = Reshape11()(x) # wavelengths x timepoints
    x = Conv1DBlockSqueezeformer(dim,conv_filter)(x)  # squeeezefilter out columns of time, might not be super effective tbh

    for _ in range(10):
        x = Reshape11()(x) # timepoints*wavelengths
        x = TransformerEncoder(wavelengths, 4, wavelengths*4)(x)
        x = Reshape11()(x) # wavelengths x timepoints
        x = Conv1DBlockSqueezeformer(dim,conv_filter)(x)  # squeeezefilter out columns of time, might not be super effective tbh

    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)
    model = tf.keras.Model(inp, x_pred)
    return model

def transformer():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    conv_filter = 15
    dim=timepoints
    x = TransformerEncoder(wavelengths, 4, wavelengths*4)(x)
    x = Reshape11()(x) # wavelengths x timepoints
    x = tf.keras.layers.Dense(283, activation='linear')(x)
    x = Reshape11()(x) # timepoints x wavelengths

    x = TransformerEncoder(wavelengths, 4, wavelengths*4)(x)
    x = Reshape11()(x) # wavelengths x timepoints
    x = tf.keras.layers.Dense(283, activation='linear')(x)
    x = Reshape11()(x) # timepoints x wavelengths

    #x = TransformerEncoder(wavelengths, 4, wavelengths*4)(x)
    #x = Reshape11()(x) # wavelengths x timepoints
    #x = tf.keras.layers.Dense(283, activation='linear')(x)
    #x = Reshape11()(x) # timepoints x wavelengths
#
    #x = TransformerEncoder(wavelengths, 4, wavelengths*4)(x)
    #x = Reshape11()(x) # wavelengths x timepoints
    #x = tf.keras.layers.Dense(75, activation='linear')(x)
    #x = Reshape11()(x) # timepoints x wavelengths

    x = TransformerEncoder(wavelengths, 4, wavelengths*4)(x)
    x = Reshape11()(x) # wavelengths x timepoints
    x = tf.keras.layers.Dense(75, activation='linear')(x)
    x = Reshape11()(x) # timepoints x wavelengths
    
    x = TransformerEncoder(wavelengths, 4, wavelengths*4)(x)
    x = Reshape11()(x) # wavelengths x timepoints
    x = tf.keras.layers.Dense(15, activation='linear')(x)

    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)
    model = tf.keras.Model(inp, x_pred)
    return model

class transf1d(tf.keras.layers.Layer):
    def __init__(self, inputDim,embed_dim, num_heads, feed_forward_dim, reshape=True):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=inputDim//num_heads)
        #self.ffn = GLUMlp(feed_forward_dim, embed_dim)
        self.ffn2 = tf.keras.layers.Dense(feed_forward_dim)
        self.reshape1 = Reshape11()
        self.reshape2 = Reshape11()
        self.reshape = reshape
    def call(self, x, training = None):
        residual = x
        x = self.att(x,x)
        x = x + residual
        if self.reshape:
            x = self.reshape1(x)
        #x = self.ffn(x)
        x = self.ffn2(x)
        #x = self.reshape2()(x)
        return x
class att1d(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim//num_heads)
    def call(self, x, training = None):
        residual = x
        x = self.att(x,x)
        x = x + residual
        return x
    
class custConv1dDepthwise(tf.keras.layers.Layer):
    def __init__(self, arr, meanInit=True):
        super().__init__()
        self.arr = arr
        self.maxSlidingWindow = max(self.arr)
        if meanInit:
            self.convArr = [tf.keras.layers.DepthwiseConv1D(kernel_size=kernelS,strides=1,padding='valid', depth_multiplier=1,activation='linear', depthwise_initializer=tf.keras.initializers.Constant(1.0 / kernelS)) for kernelS in self.arr]
        else:
            self.convArr = [tf.keras.layers.DepthwiseConv1D(kernel_size=kernelS,strides=1,padding='valid', depth_multiplier=1) for kernelS in self.arr]
    def call(self, x, training = None):
        #print(x.shape)
        outDim = x.shape[1] - self.maxSlidingWindow +1
        out=[]
        for i in range(len(self.arr)):  
            #print(x.shape)
            x0=self.convArr[i](x)
            thisOutDim = x.shape[1] - self.arr[i] +1 
            startIdx = int((thisOutDim - outDim)/2)
            x0 = x0[:,startIdx:startIdx+outDim,:]
            out.append(x0)
        x = tf.keras.layers.Concatenate(axis=-1)(out)
        #print('out',x.shape)
        return x
    
class custConv1d(tf.keras.layers.Layer):
    def __init__(self, arr, meanInit=True, filters=8):
        super().__init__()
        self.arr = arr
        self.maxSlidingWindow = max(self.arr)
        if meanInit:
            self.convArr = [tf.keras.layers.Conv1D(filters=filters, kernel_size=(kernelS), padding='valid', kernel_initializer=tf.keras.initializers.Constant(1.0 / kernelS)) for kernelS in self.arr]
        else:
            self.convArr = [tf.keras.layers.Conv1D(filters=filters, kernel_size=(kernelS), padding='valid') for kernelS in self.arr]
    def call(self, x, training = None):
        outDim = x.shape[-2] - self.maxSlidingWindow +1
        out=[]
        for i in range(len(self.arr)):  
            x0=self.convArr[i](x)
            thisOutDim = x.shape[-2] - self.arr[i] +1 
            startIdx = int((thisOutDim - outDim)/2)
            x0 = x0[:,startIdx:startIdx+outDim,:]
            out.append(x0)
        x = tf.keras.layers.Concatenate(axis=-1)(out)
        return x

 
def singleWL():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,0:1,1]
    timeP = timepoints
    kernelS = 30
    #out = [x]
    if 0:
        for i in range(4): 
            #x = LearnableRollingMean(50)  (x)
            x = tf.keras.layers.Conv1D(filters=8, kernel_size=(kernelS), padding='valid')(x)
            #x = tf.keras.layers.Conv1D(filters=2, kernel_size=(kernelS), padding='valid', kernel_initializer=tf.keras.initializers.Constant(1.0 / kernelS))(x)
            #x = tf.keras.layers.DepthwiseConv1D(kernel_size=kernelS,strides=1,padding='valid', depth_multiplier=1,activation='linear')(x)
            timeP = timeP - (kernelS-1)
            #x = transf1d(embed_dim=timeP, num_heads=4, feed_forward_dim=2*timeP)(x)
            #x = Reshape11()(x)
            #x = tf.keras.layers.AveragePooling1D(2)(x)
            #out.append(x0)

    #x = custConv1d([10,30,50])(x)
    for _ in range(3):
        x = custConv1d([30,35,40,45,50,55,60,65], meanInit=False)(x)
        #x = tf.keras.layers.Dense(24)(x)
        #x = tf.keras.layers.Conv1D(filters=8, kernel_size=(kernelS), padding='valid')(x)
        #x = custConv1d([10,30,50])(x)
        #print(x.shape)
        #x = TransformerEncoder(embed_dim=x.shape[2], num_heads=4, feed_forward_dim=x.shape[2]*2)(x)
        #x = tf.keras.layers.Dense(x.shape[2])(x)
    #x = att1d(embed_dim=outDim, num_heads=4, feed_forward_dim=2*outDim)(x)
    #x = tf.keras.layers.Dense(1000)(x)
    #x = Reshape11()(x)
    #x = tf.keras.layers.DepthwiseConv1D(kernel_size=kernelS,strides=1,padding='valid', depth_multiplier=1,activation='linear')(x)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(5)(x)
    x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model   

def cnn4converges():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([30,50], meanInit=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(2):
        x = custConv1dDepthwise([30,50], meanInit=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(283, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        #x = custConv1d([5,10,20,30,50], meanInit=False, filters=283)(x)
        #x = transf1d(inputDim=5*283,embed_dim=283,num_heads=4, feed_forward_dim=283)(x)
        #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    #x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(5)(x)
    #x = tf.keras.layers.Dense(32, activation='relu')(x)
    #x = tf.keras.layers.Dense(283, activation='relu')(x)
    x = Reshape11()(x)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(283, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model  

def cnn5converges():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([50], meanInit=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(2):
        x = custConv1dDepthwise([50], meanInit=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)

    x = Reshape11()(x)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(283, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model  

#seems to be very slow
def cnn51():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([50], meanInit=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(3):
        x = custConv1dDepthwise([50], meanInit=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(283, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x = custConv1dDepthwise([20], meanInit=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Reshape11()(x)
    #x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred[:,:,0])
    return model  

#easily fits one batch
# full dataset is harder to fit + has overfitting
def cnn52():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([20], meanInit=True)(x)
    x = tf.keras.layers.AveragePooling1D(2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(1):
        x = custConv1dDepthwise([20], meanInit=False)(x)
        x = tf.keras.layers.AveragePooling1D(2)(x)
        x = tf.keras.layers.BatchNormalization()(x)

    x = Reshape11()(x)
    x = tf.keras.layers.Dense(2, activation='relu')(x)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(283, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model   

def cnn53():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([20], meanInit=True)(x)
    x = tf.keras.layers.AveragePooling1D(2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(3):
        x = custConv1dDepthwise([20], meanInit=False)(x)
        #x = tf.keras.layers.AveragePooling1D(2)(x)
        x = tf.keras.layers.BatchNormalization()(x)

    #x = tf.keras.layers.Dense(283, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x = Reshape11()(x)
    x = tf.keras.layers.Dense(10, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(283, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model   

def cnn54():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([5], meanInit=True)(x)
    x = tf.keras.layers.AveragePooling1D(2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(3):
        x = custConv1dDepthwise([5], meanInit=False)(x)
        x = tf.keras.layers.AveragePooling1D(2)(x)
        x = tf.keras.layers.BatchNormalization()(x)

    #x = tf.keras.layers.Dense(283, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x = Reshape11()(x)
    x = tf.keras.layers.Dense(10, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(283, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model  

def cnn55():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([50], meanInit=True)(x)
    #x = tf.keras.layers.AveragePooling1D(2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    dim = timepoints - 50 +1

    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(3):
        x = transf1d(inputDim=dim,embed_dim=283,num_heads=4, feed_forward_dim=dim)(x)
        x = Reshape11()(x)
        x = custConv1dDepthwise([50], meanInit=False)(x)
        dim = dim - 50 +1
        #x = tf.keras.layers.AveragePooling1D(2)(x)
        x = tf.keras.layers.BatchNormalization()(x)

    #x = tf.keras.layers.Dense(283, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x = Reshape11()(x)
    for i in range(3):
        x = tf.keras.layers.Dense(29, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(40, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(3, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred[:,:,0])
    return model  

def cnn56():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([50], meanInit=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(2):
        x = custConv1dDepthwise([50], meanInit=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
    
    x = Reshape11()(x)
    x = tf.keras.layers.Dense(10, activation='relu')(x)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(283, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model 
    

def cnn57():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,0,:]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([50], meanInit=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    dim = timepoints - 50+1
    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(10):
        x = transf1d(inputDim=4,embed_dim=4, num_heads=4, feed_forward_dim=2*8, reshape=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = custConv1d([12], meanInit=False,filters=4*(i+1))(x)
        #x = tf.keras.layers.Dense(4, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        dim = dim -5+1
    
    #x = Reshape11()(x)
    #x = tf.keras.layers.Dense(10, activation='relu')(x)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model   


#converges on one batch, potentially on all train data, but doesn't reduce overfitting
def cnn6():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([50], meanInit=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(2):
        x = custConv1dDepthwise([50], meanInit=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)

    x = Reshape11()(x)
   #x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x = tf.keras.layers.Dense(10, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred[:,:,0])
    return model  

def cnn7():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([50], meanInit=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(2):
        x = custConv1dDepthwise([50], meanInit=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dense(283, activation='relu')(x)
    x = Reshape11()(x)
   #x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x = tf.keras.layers.Dense(10, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred[:,:,0])
    return model  
# takes forever, no real progress
def cnn8():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints - 50 +1
    #x = custConv1d([10,30,50])(x)
    x = custConv1dDepthwise([50], meanInit=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(283*5, activation='relu')(x)
    for i in range(2):
        x = custConv1dDepthwise([50], meanInit=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        timeP = timeP - 50 +1
    #x = TransformerEncoder(embed_dim=283, num_heads=4, feed_forward_dim=2*283)(x) #relations between timepoints
    x = Reshape11()(x)
    x = TransformerEncoder(embed_dim=timeP, num_heads=4, feed_forward_dim=2*timeP)(x) #relations between wavelengths
   #x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x = tf.keras.layers.Dense(3, activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred[:,:,0])
    return model  

def cnnAttentin():
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,1]
    timeP = timepoints
    kernelS = 50
    for i in range(2):       
        x = tf.keras.layers.DepthwiseConv1D(kernel_size=kernelS,strides=1,padding='valid', depth_multiplier=1,activation='linear')(x)
        #x = tf.keras.layers.DepthwiseConv1D(kernel_size=kernelS,strides=1,padding='valid', depth_multiplier=1,activation='linear', depthwise_initializer=tf.keras.initializers.Constant(1.0 / kernelS))(x)
        #x = tf.keras.layers.Conv1D(filters=283, kernel_size=(20), padding='valid')(x)
        timeP = timeP-(kernelS-1)
        out=[]
        for wl in range(283):
            x0 = x[:,:,wl:wl+1]
            x0 = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=5)(x0,x0) + x0
            x0 = Reshape11()(x0)
            x0 = GLUMlp(timeP*2,timeP)(x0) #expand time and have a gating on it
            x0 = Reshape11()(x0)
            out.append(x0)
        x = tf.keras.layers.Concatenate(axis=-1)(out)

        #x = Reshape11()(x)
        #print(timeP)
        #nreduce=30
        #x = GLUMlp(timeP*4, timeP-nreduce)(x) #rduce time dimension
        #timeP = timeP-nreduce
        #x = Reshape11()(x)

    
    
    #x = TransformerEncoder(timeP, 4, timeP*4)(x) # attention on wavelenghts doesn't work at all!
    #x = tf.keras.layers.Conv1D(filters=283, kernel_size=(20), padding='valid')(x)

    x = Reshape11()(x)
    #x = GLUMlp(timeP*2, int(timeP/2))(x)
    #x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(100)(x)
    #x = TransformerEncoder(100, 4, 200)(x)
    #x = tf.keras.layers.Dense(50)(x)
    
    x_pred = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inp, x_pred)
    return model  
    return 
#model = cnnDepthwise() 
#model = cnn2D() 
#model= squeezeformer()
#model = fcn() 
#model = cnn1() 
#model = transformer()
#model = cnnAttentin()
#model = singleWL()
model=cnn57()
#model=cnn5converges()
model.summary()

In [47]:
model.save_weights('deviations_model_conv53_2_56.weights.h5')

In [ ]:
batch_iter = iter(train_dataset)
batch=next(batch_iter)
batch1=next(batch_iter)
batch2=next(batch_iter)
out = model(batch[0])
dataset_iterator = iter(test_dataset)
test_batch1 = next(dataset_iterator)
test_batch2 = next(dataset_iterator)
batch[0].dtype ,batch[1].dtype, out.dtype,batch[0].shape ,batch[1].shape, out.shape

In [ ]:
def maeSingleWL(y_true,y_pred):
    y_true=y_true[:,0:1]
    #tf.print(y_pred.shape)
    #y_pred=tf.reduce_mean(y_pred,axis=-1)
    #tf.print(y_true.shape, y_pred.shape)
    return tf.reduce_mean(tf.math.abs(y_true-y_pred))

def mae(y_true,y_pred):
    #tf.print(y_pred, y_true)
    #y_pred=tf.reduce_mean(y_pred,axis=-1)
    #tf.print(y_true.shape, y_pred.shape)
    return tf.reduce_sum(tf.math.abs(y_true-y_pred))

maeSingleWL(batch[1],out), mae(batch[1],out)

In [7]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="C:/Users/uic33116/Documents/documents/ariel-data-challenge-2024/training_full_model/deviations_model-{epoch:02d}.weights.h5",
    save_weights_only=True,  # Set to False if you want to save the entire model
    save_freq=300 * 4,
    verbose=1
)

In [ ]:
tf.random.set_seed(42)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(#loss='mae'
              loss=maeSingleWL
              #loss='mse'#maeSingleWL#'mae'            
              #,metrics=[log_likelihood_maxScaling]
              #,metrics=['mse']
              , optimizer=optimizer)

history = model.fit(train_dataset, 
                    #batch[0],batch[1], #verbose=2,
                    #small_train,
                    #validation_data=test_dataset,
                    validation_data=test_batch1,
                    epochs=400, batch_size=batch_size,
                    #callbacks=[lr_callback]
                    )

# batch normalization essential for gradient to travel downstream!
# with batch of 12 we converge well to mae ~3.3, mse 57 after ~3000 epochs

In [ ]:
b = next(iter(small_train))
outs = model.predict(b[0])
outs,b[1][:,0:1]

In [ ]:
outs = model.predict(test_batch1[0])
outs,test_batch1[1][:,0:1]

In [ ]:
history = model.fit(#train_dataset, 
                    #batch[0],batch[1], #verbose=2,
                    small_train,
                    #validation_data=test_dataset,
                    validation_data=test_batch1,
                    epochs=100, batch_size=batch_size,
                    #callbacks=[lr_callback]
                    )

In [122]:
model.save_weights('deviationModelCnn56_5_75.weights.h5')

# investigate

In [126]:
#pred = model.predict(normData)
def calcStats(b, plot=True, display=False):
    outputs = model.predict(b[0])
    print(outputs.shape)
    pred = outputs[:,0:283]

    if display:
        print(pred[0:10:,0:2,0], b[1][0:10:,0:2])
        print(pred[0:10:,0:2,0]*maxLabels, b[1][0:10:,0:2]*maxLabels)

    mae = np.sum(np.abs(pred[:,:]-b[1])) / pred.shape[0] / pred.shape[1]
    mse = np.sum(np.abs(pred[:,:]-b[1])**2) / pred.shape[0] / pred.shape[1]
    print('mae',mae,'mse', mse)
    for i in range(outputs.shape[0]):
        mae = np.sum(np.abs(pred[i,:]-b[1][i,:])) / pred.shape[1]
        mse = np.sum(np.abs(pred[i,:]-b[1][i,:])**2) / pred.shape[1]
        print('row',i,'mae',mae,'mse', mse)

    
    fig = go.Figure()
    m = min(100, batch[0].shape[0])
    for i in range(m): #range(12):# 
        fig.add_trace(go.Scatter(y=b[1][i,:]*maxLabels,mode='markers',name=f'gt_{i}',marker=dict(size=3),visible='legendonly'))
        fig.add_trace(go.Scatter(y=pred[i,:]*maxLabels,mode='markers',name=f'pred_{i}',marker=dict(size=3)))
    fig.show()

In [ ]:
calcStats(next(iter(small_train)))

In [ ]:
calcStats(test_batch1)

In [ ]:
outputs = model.predict(batch[0])

mae = np.sum(np.abs(outputs*100*maxLabels-batch[1][:,0:1]*100*maxLabels)) / outputs.shape[0]
mse = np.sum(np.abs(outputs*100*maxLabels-batch[1][:,0:1]*100*maxLabels)**2) / outputs.shape[0]
mae1 = np.sum(np.abs(outputs*maxLabels-batch[1][:,0:1]*maxLabels)) / outputs.shape[0]
mse1 = np.sum(np.abs(outputs*maxLabels-batch[1][:,0:1]*maxLabels)**2) / outputs.shape[0]
mae,mae1, outputs, batch[1][:,0:1]

In [ ]:
outputst = model.predict(test_batch[0])

mae = np.sum(np.abs(outputst*100*maxLabels-test_batch[1][:,0:1]*100*maxLabels)) / outputs.shape[0]
mse = np.sum(np.abs(outputst*100*maxLabels-test_batch[1][:,0:1]*100*maxLabels)**2) / outputs.shape[0]
mae1 = np.sum(np.abs(outputst*maxLabels-test_batch[1][:,0:1]*maxLabels)) / outputs.shape[0]
mse1 = np.sum(np.abs(outputs*maxLabels-test_batch[1][:,0:1]*maxLabels)**2) / outputs.shape[0]
mae,mae1, outputst, test_batch[1][:,0:1]

# visualize layers

In [ ]:
batchid = 2
wl = 0
x = batch[0][batchid:batchid+1,:,:,1]  
print(x.shape)  
fig = plt.figure()
plt.plot(x[0,:,wl])
plt.title(f'input')
plt.show()

for layers in range(len(model.layers)-1):
    x = model.layers[layers+1](x)
    print(x.shape)
    
    if len(x.shape)>=3:
        if len(x.shape) == 4:
            for i in range(x.shape[-1]):
                fig = plt.figure()
                plt.plot(x[0,:,wl,i])
                plt.title(f'layer {layers+1}')
                plt.show()
        else:
            fig = plt.figure()
            if x.shape[2] == 283:
                plt.plot(x[0,:,wl])
            else:
                if x.shape[2] == 1:
                    print(x[:,wl,:], batch[1][batchid:batchid+1,0:1])
                else:
                    plt.plot(x[0,wl,:])
            plt.title(f'layer {layers+1}')
            plt.show()
    else:
        if x.shape[1] >1:
            fig=plt.figure()
            plt.plot(x[0])
            plt.title(f'layer{layers+1}')
            plt.show()
        else:
            print(x, batch[1][batchid:batchid+1,0:1])
#print(model.layers[2].get_weights())

In [ ]:
batchid = 0
wl = 0
x = batch[0][batchid:batchid+1,:,:,1]  
print(x.shape)  
fig = plt.figure()
plt.plot(x[0,:,wl])
plt.title(f'input')
plt.show()

x = x[:,:,wl:wl+1]
for layers in range(len(model.layers)-1):
    print(x.shape)
    x = model.layers[layers+1](x)
    print(x.shape)
    
    fig = plt.figure()
    if len(x.shape) == 3:
        for i in range(x.shape[2]):
            fig = plt.figure()
            plt.plot(np.reshape(x[0,:,i],(-1)))
            plt.title(f'layer {layers+1}')
        plt.show()
    else:
        plt.plot(x[0,:])
        plt.title(f'layer {layers+1}')
        plt.show()
#print(model.layers[2].get_weights())

In [ ]:
fig = plt.figure()
plt.plot(batch[0][batchid,:,wl,1]  )
plt.title(f'input')
plt.show()

In [ ]:
batch[0][batchid:batchid+1,:,wl,1]

In [ ]:
print(batch[1])
for i in range(4):
    fig=plt.figure()
    plt.plot(x[i,:,0])
    fig.show()